### Ingest drivers.json file

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType 

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                  StructField("surname", StringType(), True)])

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                     StructField("driverRef", StringType(), True),
                                     StructField("number", IntegerType(), True),
                                     StructField("code", StringType(), True),
                                     StructField("name", name_schema),
                                     StructField("dob", DateType(), True),
                                     StructField("nationality", StringType(), True),
                                     StructField("url", StringType(), True)])

In [0]:
drivers_schema = spark.read.schema(drivers_schema).json(f"{raw_folder_path}/{v_file_date}/drivers.json")


In [0]:
# display(drivers_schema)

#### Step 2 - rename columns and add new columns
1. driverId renamed to driver_id
1. driverRef renamed to driver_ref
1. ingestion date added
1. name added with concatenation of forename and surname

In [0]:
from pyspark.sql.functions import current_timestamp, concat, col, lit

In [0]:
drivers_with_columns_df = drivers_schema.withColumnRenamed("driverId", "driver_id") \
                                          .withColumnRenamed("driverRef", "driver_ref") \
                                          .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                          .withColumn("data_source", lit(v_data_source)) \
                                          .withColumn("file_date", lit(v_file_date))
                                          


In [0]:
drivers_ingestion_date_df = add_ingestion_date(drivers_with_columns_df)

In [0]:
# display(drivers_ingestion_date_df)

##### Step 3 - Drop unwantted columns
1. name.forename
1. name.surname
1. url

In [0]:
drivers_final_df = drivers_ingestion_date_df.drop(col("url"))

In [0]:
# display(drivers_final_df)

In [0]:
# drivers_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")

In [0]:
# Writing the data into a table in a database we created using spark sql -- f1_processed --Section 18
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
%sql 
SELECT * FROM f1_processed.drivers;

In [0]:
 # display(spark.read.parquet(f"{processed_folder_path}/drivers"))

In [0]:
dbutils.notebook.exit("Success")